In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report,
    roc_auc_score, f1_score, precision_recall_curve
)
from xgboost import XGBClassifier

# SHAP kurulumu kontrol
try:
    import shap
    SHAP_AVAILABLE = True
    print("✅ SHAP kütüphanesi mevcut")
except ImportError:
    SHAP_AVAILABLE = False
    print("⚠️ SHAP kütüphanesi yüklü değil. Feature importance XGBoost'tan alınacak.")
    print("   Yüklemek için: pip install shap")

import warnings
warnings.filterwarnings('ignore')

# Paths
DATA_PATH = '../data/train_numeric.csv'
MODEL_PATH = '../models/bosch_quality_classifier.pkl'
FEATURES_PATH = '../models/feature_columns.pkl'
CONFIG_PATH = '../models/model_config.pkl'

print("\n📦 Kütüphaneler yüklendi")

## 1. Veri ve Model Yükleme

In [ ]:
# Model ve konfigürasyon yükle
with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

with open(FEATURES_PATH, 'rb') as f:
    feature_columns = pickle.load(f)

with open(CONFIG_PATH, 'rb') as f:
    config = pickle.load(f)

print(f"✅ Model yüklendi")
print(f"📊 Feature sayısı: {len(feature_columns)}")
print(f"🎯 Optimal threshold: {config.get('best_threshold', 0.5)}")

In [ ]:
# Test verisi hazırla
SAMPLE_SIZE = 100_000
print("\n📊 Veri yükleniyor...")
df = pd.read_csv(DATA_PATH, nrows=SAMPLE_SIZE)

y = df['Response']
X = df.drop(['Response', 'Id'], axis=1)

# Eksik veri temizleme
missing_ratio = X.isnull().sum() / len(X)
cols_to_drop = missing_ratio[missing_ratio > 0.90].index.tolist()
X_cleaned = X.drop(columns=cols_to_drop)
X_cleaned = X_cleaned.fillna(-999)

# Eğitimde kullanılan sütunları seç
common_cols = [c for c in feature_columns if c in X_cleaned.columns]
X_final = X_cleaned[common_cols]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✅ Test seti: {X_test.shape}")

## 2. SHAP Analizi - Model Açıklanabilirliği

SHAP (SHapley Additive exPlanations):
- Her feature'ın tahmine katkısını ölçer
- Pozitif değer: Hatalı parça olasılığını artırıyor
- Negatif değer: Sağlam parça olasılığını artırıyor

In [ ]:
if SHAP_AVAILABLE:
    print("🔍 SHAP değerleri hesaplanıyor...")
    print("⏳ Bu işlem birkaç dakika sürebilir...\n")
    
    # SHAP explainer oluştur (TreeExplainer XGBoost için optimize)
    explainer = shap.TreeExplainer(model)
    
    # Küçük bir örneklem üzerinde hesapla (hız için)
    sample_size = min(1000, len(X_test))
    X_sample = X_test.iloc[:sample_size]
    
    shap_values = explainer.shap_values(X_sample)
    print(f"✅ SHAP değerleri hesaplandı ({sample_size} örnek)")
else:
    print("⚠️ SHAP mevcut değil. XGBoost feature importance kullanılacak.")

In [ ]:
if SHAP_AVAILABLE:
    # SHAP Summary Plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_sample, plot_type="bar", max_display=20, show=False)
    plt.title('SHAP Feature Importance (Top 20)', fontsize=14)
    plt.tight_layout()
    plt.savefig('../docs/shap_importance.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✅ SHAP grafiği kaydedildi: docs/shap_importance.png")
else:
    # XGBoost Feature Importance
    feature_importance = pd.DataFrame({
        'feature': common_cols,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    top_20 = feature_importance.head(20)
    plt.barh(range(len(top_20)), top_20['importance'], color='steelblue')
    plt.yticks(range(len(top_20)), top_20['feature'])
    plt.gca().invert_yaxis()
    plt.xlabel('Importance')
    plt.title('XGBoost Feature Importance (Top 20)')
    plt.tight_layout()
    plt.savefig('../docs/xgb_feature_importance.png', dpi=150)
    plt.show()

In [ ]:
if SHAP_AVAILABLE:
    # SHAP Beeswarm Plot (detaylı görünüm)
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_sample, max_display=15, show=False)
    plt.title('SHAP Değerleri Dağılımı', fontsize=14)
    plt.tight_layout()
    plt.savefig('../docs/shap_beeswarm.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n📊 SHAP Yorumu:")
    print("- Kırmızı noktalar: Yüksek feature değerleri")
    print("- Mavi noktalar: Düşük feature değerleri")
    print("- Sağa doğru: Hatalı parça olasılığını artırıyor")
    print("- Sola doğru: Sağlam parça olasılığını artırıyor")

## 3. Confusion Matrix Maliyet Analizi

### Maliyet Tanımları:
- **True Negative (TN):** Parça sağlam, model "sağlam" dedi → ✅ Sorun yok
- **True Positive (TP):** Parça hatalı, model "hatalı" dedi → ✅ Başarılı tespit
- **False Positive (FP):** Parça sağlam, model "hatalı" dedi → ⚠️ Gereksiz inceleme maliyeti
- **False Negative (FN):** Parça hatalı, model "sağlam" dedi → 🚨 **KRİTİK!** Müşteriye hatalı ürün

In [ ]:
# Tahminler
best_threshold = config.get('best_threshold', 0.5)
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= best_threshold).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("="*60)
print("📊 CONFUSION MATRIX ANALİZİ")
print("="*60)
print(f"\nThreshold: {best_threshold}")
print(f"\n{'':15} {'Tahmin: Sağlam':>15} {'Tahmin: Hatalı':>15}")
print(f"{'Gerçek: Sağlam':15} {tn:>15,} {fp:>15,}")
print(f"{'Gerçek: Hatalı':15} {fn:>15,} {tp:>15,}")

In [ ]:
# Maliyet Hesaplama (Varsayımsal değerler)
print("\n" + "="*60)
print("💰 MALİYET ANALİZİ (Varsayımsal)")
print("="*60)

# Birim maliyetler (örnek değerler)
COST_FP = 10    # Gereksiz inceleme maliyeti ($)
COST_FN = 500   # Müşteriye hatalı ürün gönderme maliyeti ($)
COST_INSPECTION = 5  # Normal kalite kontrol maliyeti ($)

# Maliyet hesabı
total_fp_cost = fp * COST_FP
total_fn_cost = fn * COST_FN
total_cost = total_fp_cost + total_fn_cost

# AI olmadan maliyet (tüm hatalı ürünler müşteriye gider varsayımı)
total_defects = tp + fn
cost_without_ai = total_defects * COST_FN

# Tasarruf
savings = cost_without_ai - total_cost
savings_pct = (savings / cost_without_ai * 100) if cost_without_ai > 0 else 0

print(f"\n📋 Birim Maliyetler:")
print(f"   - False Positive (gereksiz inceleme): ${COST_FP}")
print(f"   - False Negative (kaçan hata): ${COST_FN}")

print(f"\n📊 Test Seti Sonuçları ({len(y_test):,} parça):")
print(f"   - Yakalanan hatalar (TP): {tp:,} parça")
print(f"   - Kaçan hatalar (FN): {fn:,} parça")
print(f"   - Yanlış alarmlar (FP): {fp:,} parça")

print(f"\n💵 Maliyet Özeti:")
print(f"   - False Positive maliyeti: ${total_fp_cost:,.0f}")
print(f"   - False Negative maliyeti: ${total_fn_cost:,.0f}")
print(f"   - TOPLAM MALİYET: ${total_cost:,.0f}")

print(f"\n🎯 AI Değeri:")
print(f"   - AI olmadan maliyet: ${cost_without_ai:,.0f}")
print(f"   - AI ile maliyet: ${total_cost:,.0f}")
print(f"   - TASARRUF: ${savings:,.0f} ({savings_pct:.1f}%)")

In [ ]:
# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Confusion Matrix Heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Sağlam (0)', 'Hatalı (1)'],
            yticklabels=['Sağlam (0)', 'Hatalı (1)'])
axes[0].set_xlabel('Tahmin', fontsize=12)
axes[0].set_ylabel('Gerçek', fontsize=12)
axes[0].set_title(f'Confusion Matrix\n(Threshold: {best_threshold})', fontsize=14)

# Maliyet annotation
axes[0].text(0.5, -0.15, f'FP Maliyeti: ${total_fp_cost:,.0f}', 
             transform=axes[0].transAxes, ha='center', fontsize=10, color='orange')
axes[0].text(0.5, -0.22, f'FN Maliyeti: ${total_fn_cost:,.0f}', 
             transform=axes[0].transAxes, ha='center', fontsize=10, color='red')

# 2. Maliyet Karşılaştırması
costs = ['AI Olmadan', 'AI ile', 'Tasarruf']
values = [cost_without_ai, total_cost, savings]
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']

bars = axes[1].bar(costs, values, color=colors, edgecolor='black')
axes[1].set_ylabel('Maliyet ($)', fontsize=12)
axes[1].set_title('Maliyet Karşılaştırması', fontsize=14)

# Değer etiketleri
for bar, val in zip(bars, values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100,
                f'${val:,.0f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../docs/cost_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Maliyet analizi kaydedildi: docs/cost_analysis.png")

## 4. Threshold Trade-off Analizi

Eşik değerini düşürmek:
- ✅ Daha fazla hata yakalar (Recall artar)
- ❌ Daha fazla yanlış alarm (Precision düşer)

In [ ]:
# Farklı threshold değerleri için maliyet analizi
thresholds = np.arange(0.1, 0.9, 0.05)
results = []

for thresh in thresholds:
    y_pred_t = (y_pred_proba >= thresh).astype(int)
    cm_t = confusion_matrix(y_test, y_pred_t)
    tn_t, fp_t, fn_t, tp_t = cm_t.ravel()
    
    cost_t = fp_t * COST_FP + fn_t * COST_FN
    recall_t = tp_t / (tp_t + fn_t) if (tp_t + fn_t) > 0 else 0
    precision_t = tp_t / (tp_t + fp_t) if (tp_t + fp_t) > 0 else 0
    
    results.append({
        'threshold': thresh,
        'total_cost': cost_t,
        'fp_cost': fp_t * COST_FP,
        'fn_cost': fn_t * COST_FN,
        'recall': recall_t,
        'precision': precision_t,
        'fp': fp_t,
        'fn': fn_t
    })

results_df = pd.DataFrame(results)
optimal_idx = results_df['total_cost'].idxmin()
optimal_thresh = results_df.loc[optimal_idx, 'threshold']

print(f"💰 Maliyet açısından optimal threshold: {optimal_thresh:.2f}")
print(f"   Minimum maliyet: ${results_df.loc[optimal_idx, 'total_cost']:,.0f}")

In [ ]:
# Trade-off görselleştirmesi
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Maliyet vs Threshold
axes[0].plot(results_df['threshold'], results_df['total_cost'], 'b-', linewidth=2, label='Toplam Maliyet')
axes[0].plot(results_df['threshold'], results_df['fp_cost'], 'g--', linewidth=1.5, label='FP Maliyeti')
axes[0].plot(results_df['threshold'], results_df['fn_cost'], 'r--', linewidth=1.5, label='FN Maliyeti')
axes[0].axvline(x=optimal_thresh, color='purple', linestyle=':', label=f'Optimal ({optimal_thresh:.2f})')
axes[0].axvline(x=best_threshold, color='orange', linestyle=':', label=f'Kullanılan ({best_threshold:.2f})')
axes[0].set_xlabel('Threshold', fontsize=12)
axes[0].set_ylabel('Maliyet ($)', fontsize=12)
axes[0].set_title('Threshold vs Maliyet', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Precision-Recall Trade-off
axes[1].plot(results_df['threshold'], results_df['recall'], 'b-', linewidth=2, label='Recall (Hata Yakalama)')
axes[1].plot(results_df['threshold'], results_df['precision'], 'g-', linewidth=2, label='Precision (Doğruluk)')
axes[1].axvline(x=best_threshold, color='orange', linestyle=':', label=f'Kullanılan ({best_threshold:.2f})')
axes[1].set_xlabel('Threshold', fontsize=12)
axes[1].set_ylabel('Skor', fontsize=12)
axes[1].set_title('Precision-Recall Trade-off', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/threshold_tradeoff.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Trade-off analizi kaydedildi: docs/threshold_tradeoff.png")

## 5. İş Önerileri ve Sonuç

In [ ]:
# Final metrikleri hesapla
final_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
final_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
final_auc = roc_auc_score(y_test, y_pred_proba)
final_f1 = f1_score(y_test, y_pred)

print("="*70)
print("📋 FINAL RAPOR: BOSCH KALİTE TAHMİN MODELİ")
print("="*70)

print("\n🎯 MODEL PERFORMANSI:")
print(f"   - AUC-ROC: {final_auc:.4f}")
print(f"   - F1-Score: {final_f1:.4f}")
print(f"   - Precision: {final_precision:.4f}")
print(f"   - Recall: {final_recall:.4f}")
print(f"   - Optimal Threshold: {best_threshold}")

print("\n💡 İŞ ÖNERİLERİ:")
print(f"")
print(f"   1. HATA YAKALAMA:")
print(f"      Model, toplam {tp+fn:,} hatalı parçanın {tp:,} tanesini ({final_recall*100:.1f}%) yakalıyor.")
print(f"      Kaçan hatalar: {fn:,} parça")
print(f"")
print(f"   2. YANLIŞ ALARM:")
print(f"      {fp:,} sağlam parça gereksiz incelemeye gönderiliyor.")
print(f"      Bu, ekstra işçilik maliyeti demek ama müşteri memnuniyetini korur.")
print(f"")
print(f"   3. MALİYET TASARRUFU:")
print(f"      AI sistemi yıllık ${savings:,.0f} tasarruf sağlayabilir.")
print(f"      ({savings_pct:.1f}% maliyet azalması)")
print(f"")
print(f"   4. ÖNCELİKLİ İSTASYONLAR:")
print(f"      Feature importance'a göre en kritik sensörlere bakım önceliği verilmeli.")

print("\n" + "="*70)
print("✅ SONUÇ: Model üretime hazır. Pilot test önerilir.")
print("="*70)

In [ ]:
# Final pipeline bilgilerini kaydet
final_report = {
    'model_metrics': {
        'auc_roc': final_auc,
        'f1_score': final_f1,
        'precision': final_precision,
        'recall': final_recall,
        'threshold': best_threshold
    },
    'confusion_matrix': {
        'tn': int(tn),
        'fp': int(fp),
        'fn': int(fn),
        'tp': int(tp)
    },
    'cost_analysis': {
        'fp_unit_cost': COST_FP,
        'fn_unit_cost': COST_FN,
        'total_fp_cost': total_fp_cost,
        'total_fn_cost': total_fn_cost,
        'total_cost': total_cost,
        'savings': savings,
        'savings_percentage': savings_pct
    },
    'business_recommendations': [
        f"Model {final_recall*100:.1f}% hata yakalama oranına sahip",
        f"Yıllık tahmini tasarruf: ${savings:,.0f}",
        "Pilot test için önerilen başlangıç: Tek üretim hattı",
        "Threshold ayarı iş gereksinimlerine göre yapılmalı"
    ]
}

with open('../models/final_report.pkl', 'wb') as f:
    pickle.dump(final_report, f)

print("\n✅ Final rapor kaydedildi: models/final_report.pkl")

---

## Özet

### Bu Notebook'ta Yapılanlar:
1. ✅ SHAP/XGBoost ile feature importance analizi
2. ✅ Confusion matrix maliyet hesabı
3. ✅ Threshold trade-off analizi
4. ✅ İş önerileri ve ROI hesabı

### Üretilen Dosyalar:
- `docs/shap_importance.png` (veya `xgb_feature_importance.png`)
- `docs/cost_analysis.png`
- `docs/threshold_tradeoff.png`
- `models/final_report.pkl`

### Sonraki Adımlar:
- Streamlit UI üzerinden model testi
- Gerçek üretim verisi ile pilot test
- Modelin periyodik yeniden eğitimi